# ***Distribución ESTACIONAMIENTOS***

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "pachuca" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_pachuca',
 'df_alfa_agosto_2024_pachuca',
 'df_alfa_febrero_2024_pachuca',
 'df_alfa_julio_2024_pachuca',
 'df_alfa_junio_2024_pachuca',
 'df_alfa_marzo_2024_pachuca',
 'df_alfa_mayo_2024_pachuca',
 'df_alfa_q_feb_2023_pachuca',
 'df_alfa_q_jul_2023_pachuca',
 'df_alfa_q_jun_2023_pachuca',
 'df_alfa_q_mar_2023_pachuca',
 'df_alfa_q_nov_2022_pachuca',
 'df_alfa_q_oct_2022_pachuca',
 'df_alfa_q_sep_2023_pachuca']

In [3]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Categoria':'categoria','Estacionamiento':'estacionamiento','Estacionamientos':'estacionamiento','id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido','Precio':'precio'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [4]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','estacionamiento','precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [5]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

In [6]:
for df in dataframes_list:    
    df = df.drop(columns=['categoria'])
    # Asignar la categoría real a cada registro
    df['categoria'] = df['precio'].apply(asignar_categoria)

In [7]:
def procesar_dataframes(dataframes_list, nombres_df_filtrados):
    for i, df in enumerate(dataframes_list):
        # Crear una nueva columna con la primera letra de la categoría
        df['segmento'] = df['categoria'].str[0]
        df.drop(columns=['categoria'], inplace=True)

        # Clasificar 'baño_total' en rangos específicos
        bins = [1, 2, 3, 4, 5, 6, float('inf')]
        labels = ['1', '2', '3', '4', '5', '6']
        df['estacionamiento'] = pd.cut(df['estacionamiento'], bins=bins, labels=labels, right=False)

        # Agrupar y contar los datos
        tabla = df.groupby(['segmento', 'estacionamiento']).size()

        # Imprimir el nombre del DataFrame y la tabla resultante
        print(f"Nombre del DataFrame: {nombres_df_filtrados[i]}")
        print(tabla)

# Procesar los DataFrames de la lista
procesar_dataframes(dataframes_list, nombres_df_filtrados)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11068\3394540407.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11068\3394540407.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['categoria'], inplace=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11068\3394540407.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

Nombre del DataFrame: df_alfa_abril_2024_pachuca
segmento  estacionamiento
A         1                   29
          2                  104
          3                    0
          4                    0
          5                    0
          6                    0
B         1                   45
          2                  152
          3                    0
          4                    0
          5                    0
          6                    0
C         1                   54
          2                  193
          3                    0
          4                    0
          5                    0
          6                    0
D         1                   83
          2                  253
          3                    0
          4                    0
          5                    0
          6                    0
E         1                   27
          2                  184
          3                    0
          4                    0
 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11068\3394540407.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['estacionamiento'] = pd.cut(df['estacionamiento'], bins=bins, labels=labels, right=False)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11068\3394540407.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla = df.groupby(['segmento', 'estacionamiento']).size()
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11068\3394540407.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [8]:
# Crear una nueva función para sumar las tablas de todos los DataFrames
def sumar_tablas(dataframes_list):
    # Inicializar una tabla vacía para almacenar la suma
    tabla_suma = None
    
    # Iterar sobre cada DataFrame en la lista
    for df in dataframes_list:
        # Agrupar y contar los datos en el DataFrame actual
        tabla_actual = df.groupby(['segmento', 'estacionamiento'],observed=False).size().unstack(fill_value=0)
        
        # Sumar la tabla actual al acumulador (tabla_suma)
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    # Imprimir la tabla resultante de la suma
    print("Tabla general de frecuencia:")
    print(tabla_suma)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
sumar_tablas(dataframes_list)

Tabla general de frecuencia:
estacionamiento       1       2      3      4      5     6
segmento                                                  
A                 574.0  1375.0  219.0  167.0   16.0   9.0
B                 969.0  2065.0  319.0  228.0    2.0   8.0
C                1087.0  1850.0  317.0  117.0    5.0   9.0
D                1606.0  2150.0  200.0    7.0    1.0   5.0
E                1115.0   795.0   66.0   13.0   11.0   0.0
L                  72.0    70.0    0.0   35.0   43.0   0.0
S                 944.0  2011.0  509.0  257.0  338.0  85.0


In [9]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9','#E50CB6','#4225CF']
orden_segmentos=["E", "D", "C", "B","A", "S", "L", ]

def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
         # Aplica el orden a la columna 'segmento'
        df['segmento'] = pd.Categorical(df['segmento'], categories=orden_segmentos, ordered=True)
        
        tabla_actual = df.groupby(['estacionamiento', 'segmento'],observed=False).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_bar_estacionamientos', carpeta='assets/graficas')
    
    return fig

def graficar_tabla(tabla):
    segmentos = tabla.index
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, segmento in enumerate(segmentos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=banos,
            y=tabla.loc[segmento],
            name=segmento,
            marker_color=color
        ))
    
    fig.update_layout(
        #title="M2 de Terreno",
        #xaxis_title="Rango de M2",
         yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        legend_title="Estacionamientos",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    )
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    #pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11068\3575200082.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = pd.Categorical(df['segmento'], categories=orden_segmentos, ordered=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11068\3575200082.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = pd.Categorical(df['segmento'], categories=orden_segmentos, ordered=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_11068\3575200082.py:9: SettingWithCopyWarning: 
A value is try

Tabla general de frecuencia:
segmento            E     D     C     B     A     S   L
estacionamiento                                        
1                1115  1606  1087   969   574   944  72
2                 795  2150  1850  2065  1375  2011  70
3                  66   200   317   319   219   509   0
4                  13     7   117   228   167   257  35
5                  11     1     5     2    16   338  43
6                   0     5     9     8     9    85   0


In [10]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2', '#F79CB9', '#E50CB6', '#4225CF']

def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['estacionamiento', 'segmento'],observed=False).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_pie_estacionamientos', carpeta='assets/graficas')
    
    return fig

def graficar_tabla(tabla):
    # Sumar los valores para cada estacionamiento
    suma_estacionamientos = tabla.sum(axis=1)
    
    fig = go.Figure(data=[go.Pie(
        labels=suma_estacionamientos.index,
        values=suma_estacionamientos.values,
        marker=dict(colors=colores[:len(suma_estacionamientos)]),
        #hole=.3  # Si deseas un gráfico de pastel tipo "donut"
    )])
    
    fig.update_layout(
        #title="Distribución de Estacionamientos por Segmento",
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    )
    
    fig.show()
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

fig = sumar_tablas(dataframes_list)

Tabla general de frecuencia:
segmento            E     D     C     B     A     S   L
estacionamiento                                        
1                1115  1606  1087   969   574   944  72
2                 795  2150  1850  2065  1375  2011  70
3                  66   200   317   319   219   509   0
4                  13     7   117   228   167   257  35
5                  11     1     5     2    16   338  43
6                   0     5     9     8     9    85   0


# DIRECTOS

In [4]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

data = {
    'Segmento': ['E', 'D', 'C', 'B', 'A', 'S', 'L'],
    '1': [1314,143,177,130,93,64,266],
    '2': [118,52,43,28,39,17,0],
    '3': [314,0,0,0,0,0,0],    
}
# Convertir a DataFrame
dataframes_list = pd.DataFrame(data)

In [10]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9','#E50CB6','#4225CF']
orden_segmentos=["E", "D", "C", "B","A", "S", "L"]

def graficar_tabla(tabla):
    segmentos = tabla.index
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, estacionamiento in enumerate(banos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=segmentos,
            y=tabla.loc[:, estacionamiento],
            name=estacionamiento,
            marker_color=color
        ))
    
    fig.update_layout(
        #title="M2 de Terreno",
        #xaxis_title="Rango de M2",
         yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        legend_title="Estacionamientos",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    )
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
    #pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

# Llamar a la función graficar_tabla con la tabla como argumento
data = {
    'Segmento': ['E', 'D', 'C', 'B', 'A', 'S', 'L'],
    '1': [64,93,130,177,143,1314,266],
    '2': [17,39,28,43,52,118,0],
    '3': [0,0,0,0,0,314,0],    
}
df = pd.DataFrame(data)
df = df.set_index('Segmento')
fig = graficar_tabla(df)
# Guardar la gráfica como archivo HTML
guardar_grafico_como_html(fig, 'g_bar_estacionamientos', carpeta='assets/graficas')

In [14]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9','#E50CB6','#4225CF']
orden_segmentos=["E", "D", "C", "B","A", "S", "L"]

def graficar_tabla(tabla):
    # Calcular el porcentaje de cada estacionamiento
    porcentajes = tabla.sum() / tabla.sum().sum()
    
    fig = go.Figure(data=[go.Pie(
        labels=tabla.columns,
        values=porcentajes,
        marker=dict(colors=colores[:len(tabla.columns)]),
    )])
    
    fig.update_layout(
        #title="Distribución de Estacionamientos",
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    )
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
    #pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

# Llamar a la función graficar_tabla con la tabla como argumento
data = {
    '1': [64,93,130,177,143,1314,266],
    '2': [17,39,28,43,52,118,0],
    '3': [0,0,0,0,0,314,0],    
}
df = pd.DataFrame(data)
fig = graficar_tabla(df)
# Guardar la gráfica como archivo HTML
guardar_grafico_como_html(fig, 'g_pie_estacionamientos', carpeta='assets/graficas')